# Ejercicio 9: Uso de la API de Google Gemini

En este ejercicio vamos a aprender a utilizar la API de OpenAI

## 1. Uso básico

El siguiente código sirve para conectarse con la API de Google Gemini de forma básica

In [2]:
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()
api_key = os.getenv('Gemini_API_KEY')

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Gemini_API_KEY")

In [32]:
from google import genai

client = genai.Client(api_key=secret_value_0)

response = client.models.generate_content(
    model="gemini-3-flash-preview",
    contents="Dime el nombre y apellido del mejor jugador del mundo"
)

print(response.text)

Esta es una de las preguntas que más debate genera en el deporte, ya que depende de si hablas de la historia completa o de la actualidad. Aquí tienes las respuestas más aceptadas:

1.  **Por palmarés e historia:** Para la mayoría de los expertos, organismos oficiales y aficionados, el mejor de la historia es **Lionel Messi**. Ha ganado 8 Balones de Oro, el Mundial de Catar 2022 y es el jugador con más títulos en la historia del fútbol.
2.  **Por competitividad y goles:** Muchos consideran que el mejor es **Cristiano Ronaldo**, por ser el máximo goleador histórico del fútbol profesional y su dominio en la Champions League.
3.  **En la actualidad (2024):** Si te refieres a quién está en la cima hoy en día, nombres como **Kylian Mbappé**, **Erling Haaland** o **Vinícius Júnior** suelen encabezar las listas de los mejores del momento.

**Resumen:** Si solo puedes elegir uno basado en premios oficiales, el nombre es **Lionel Messi**.


## 2. Retrieval

### 2.1 Cargo el corpus de 20 News Groups

In [6]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
newsgroupsdocs = newsgroups.data

In [7]:
import pandas as pd

df = pd.DataFrame(newsgroupsdocs, columns=['text'])
df

,text
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...
...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
18842,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...


### 2.2 Transformo a embeddings

In [8]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re
df = df.dropna(subset=["text"]).reset_index(drop=True)

def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)
df.head()

,text,text_norm
0,\n\nI am sure some bashers of Pens fans are pr...,I am sure some bashers of Pens fans are pretty...
1,My brother is in the market for a high-perform...,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...,Finally you said what you dream about. Mediter...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,Think! It's the SCSI card doing the DMA transf...
4,1) I have an old Jasmine drive which I cann...,1) I have an old Jasmine drive which I cannot ...


In [9]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end].strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in tqdm(df.iterrows(), total=len(df), desc="Fragmentando textos"):
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": i,
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
print(f"Total de chunks generados: {len(chunks_df)}")

Fragmentando textos:   0%|          | 0/18846 [00:00<?, ?it/s]

Total de chunks generados: 38871


In [10]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"
model = SentenceTransformer(MODEL_NAME)

passages = ["passage: " + t for t in chunks_df["text"].tolist()]

embeddings = model.encode(
    passages,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

print("Chunks:", len(chunks_df))
print("Embeddings:", embeddings.shape)

2026-01-09 02:43:45.814188: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767926626.144029      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767926626.242906      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767926627.097711      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767926627.097751      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767926627.097754      55 computation_placer.cc:177] computation placer alr

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/608 [00:00<?, ?it/s]

Chunks: 38871
Embeddings: (38871, 768)


### 2.3 Creo una query y hago la búsqueda

In [26]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

querry = embed_query("neural networks")

In [27]:
import faiss
import numpy as np

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(querry, k=10)

In [28]:
passages[I[0][0]]

"passage: I'm sure there are many people who work with neural networks and read this newsgroup. Please tell Kevin what you've achieved, and what you expect. Indeed. I think dualism is a non-solution, or, as Dennett recently put it, a dead horse. Petri"

In [29]:
def get_top_k_indices(query_vec, corpus_embeddings, k=5):
    similarities = np.dot(corpus_embeddings, query_vec.T).flatten()
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    
    return top_k_indices, similarities[top_k_indices]

In [31]:
def responder_con_rag(pregunta, k=5):

    qv = embed_query(pregunta)

    ind, sc = get_top_k_indices(qv, embeddings, k=k)

    try:
        ind = list(ind)
    except:
        pass

    txts = chunks_df.iloc[ind]["text"].tolist()

    ctx = ""
    for t in txts:
        ctx += str(t) + "\n---\n"

    prompt_final = ""
    prompt_final += "Eres un asistente experto. Usa el CONTEXTO para responder.\n"
    prompt_final += "Si no esta en el contexto di que no sabes.\n\n"
    prompt_final += "CONTEXTO:\n"
    prompt_final += ctx
    prompt_final += "\nPREGUNTA:\n"
    prompt_final += str(pregunta)
    prompt_final += "\n\nRESPUESTA:\n"

    r = client.models.generate_content(
        model="gemini-3-flash-preview",
        contents=prompt_final
    )

    return r.text, txts


mi_pregunta = "neural networks"
respuesta, fuentes = responder_con_rag(mi_pregunta)

print("=== RESPUESTA DE GEMINI ===")
print(respuesta)
print("\n=== FUENTES UTILIZADAS (TOP 1) ===")
print(fuentes[0])

=== RESPUESTA DE GEMINI ===
Basado en el contexto proporcionado, las **redes neuronales** (neural networks) se mencionan en los siguientes ámbitos:

*   **Aplicaciones científicas:** Se utilizan para la clasificación de proteínas ("Protein Classification using Neural Networks") y la clasificación de secuencias moleculares ("Neural Networks for Molecular Sequence Classification"). También se emplean en el **análisis de formas** mediante conceptos de minimización de energía.
*   **Publicaciones y eventos:** 
    *   Existe una serie anual de libros titulada *"Progress in Neural Networks"* de Ablex Publishing Corporation, la cual solicitó artículos para un volumen especial sobre análisis de formas. 
    *   Se mencionan presentaciones académicas sobre el tema programadas para julio de 1993.
*   **Hardware y construcción:** Un usuario consulta sobre la posibilidad de construir circuitos de redes neuronales con componentes electrónicos básicos (comprados en tiendas como Radio Shack) que no 